In [14]:
print("Imports...")

!sudo apt install -y fluidsynth
!pip install --upgrade pyfluidsynth
!pip install pretty_midi

import fluidsynth
import glob
import itertools
import music21
import numpy as np
import pathlib
import pandas as pd
import pickle
import pretty_midi
import tensorflow as tf

from IPython import display
from matplotlib import pyplot as plt
from music21 import converter, note, chord
from sklearn.model_selection import train_test_split

Imports...
Reading package lists... Done
Building dependency tree       
Reading state information... Done
fluidsynth is already the newest version (2.1.1-2).
0 upgraded, 0 newly installed, 0 to remove and 23 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [15]:
seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)
_SAMPLING_RATE = 16000

Pobieramy dataset maestro zawierający utwory MIDI.

In [16]:
data_dir = pathlib.Path('data/maestro-v3.0.0')
if not data_dir.exists():
  tf.keras.utils.get_file(
      'maestro-v3.0.0-midi.zip',
      origin='https://storage.googleapis.com/magentadata/datasets/maestro/v3.0.0/maestro-v3.0.0-midi.zip',
      extract=True,
      cache_dir='.', cache_subdir='data',
  )

In [17]:
filenames = glob.glob(str(data_dir/'**/*.mid*'))
print('Number of files:', len(filenames))

Number of files: 1276


Tworzymy listę list "notes", którą zapiszemy do pliku za pomocą biblioteki pickle (to kwestia optymalizacji czasowej, proces obróbki 1276 plików MIDI trwa bardzo dużo czasu, nawet kilkanaście godzin).

W przypadku gdy w danym utworze natrafimy na nutę to zapisujemy jej wartość stringową do listy, jeśli natrafimy na akord to zamieniamy go na "sumę stringową" nut: np. 'E5+C4+D5'.

Zatem nasza wartość notes dla przykładu ma wartość: [ [C4, E3, G#3, E5+C4+D5, ...], [...], [...], ... ], gdzie notes[i] określa listę nut/akordów dla pliku o indeksie 'i' ze zbioru Maestro.

In [25]:
notes = []
d = filenames[0:500]
all = len(d)

for count, f in enumerate(d):
    n = []
    midi = converter.parse(f)
    midi_notes = midi.flat.notes

    for c, element in enumerate(midi_notes):
        if isinstance(element, note.Note):
            n.append(str(element.pitch))

        elif isinstance(element, chord.Chord):
            n.append("+".join(str(n) for n in element.normalOrder))


    notes.append(n)
    print(f"f: {count} / {all}")

W tym i pozostałych przykładach, posługuję się biblioteką pickle do zapisu zawartości zmiennych - przyczyną tego są kwestie pamięci RAM.

In [26]:
with open("data/list_of_notes" , "wb") as file:
    pickle.dump(notes, file)

Finalnie postanowiłem ograniczyć się do prawie połowy datasetu Maestro (500 plików). Są to dane zdecydowanie wystarczające na potrzeby tego projektu, szczególnie biorąc pod uwagę, że każdy utwór zawiera w sobie średnio kilka tysięcy elementów (nut oraz akordów).

In [19]:
with open("data/list_of_notes_filenames[0_500]" , "rb") as file:
    notes = pickle.load(file)

print(len(notes))

500


In [3]:
notes_len = np.array([len(i) for i in notes])
min_len = np.min(notes_len)
max_len = np.max(notes_len)
avg_len = np.average(notes_len)

print(min_len, max_len, avg_len)

474 15197 4154.682


Zauważmy, że liczba unikalnych nut (wraz z akordami) wynosi 1335:

In [4]:
all_notes = list(itertools.chain.from_iterable(notes))
unique_notes = len(set(all_notes))
print(unique_notes)

1335


Tworzymy określenie relacji nuta <-> liczba, chcemy docelowo mieć mechanizm do zmiany nuty na liczbę i odwrotnie.

In [5]:
names = sorted(set(all_notes))
note_to_int = dict((element, idx) for idx, element in enumerate(names))
int_to_note = {idx:element for element, idx in note_to_int.items()}

assert len(note_to_int) == unique_notes
assert len(int_to_note) == unique_notes

In [20]:
with open("data/note_to_int" , "wb") as file:
    pickle.dump(note_to_int, file)

with open("data/int_to_note" , "wb") as file:
    pickle.dump(int_to_note, file)

In [6]:
def map_note_to_int(note):
  return note_to_int[note]

def map_int_to_note(note):
  return int_to_note[note]

np_note_to_int = np.vectorize(map_note_to_int)
np_int_to_note = np.vectorize(map_int_to_note)

Zamieniamy nuty z naszego wstępnego datasetu na liczby:

In [21]:
int_notes = [list(np_note_to_int(np.array(i))) for i in notes]

Tworzymy interpretację binarną liczbowych nut: dla każdej nuty tworzymy wektor zer oraz jedynki, która jest umiejscowiona na pozycji note_to_int[nuta-1]. Stąd np. dla liczby nutowej 138, wektor będzie miał postać l =  [0, 0, 0, ..., 1, 0, 0, ...], gdzie l[137] = 1.

In [8]:
def convert_to_bin(list_int_notes, unique=unique_notes):
  result_data = []
  for midi in list_int_notes:
    temp = []
    for i in midi:
      h = [0.0] * unique
      if i > 0:
        h[i-1] = 1.0
      else:
        h[0] = 1.0
      temp.append(h)
    result_data.append(temp)
  
  return result_data

In [9]:
data = convert_to_bin(int_notes)

Tworzymy zbiór X oraz predykcję (Y), długość każdego podzbioru X ustalamy domyślnie na 20 - daje to sensowne wyniki, sugestię co do wyboru takiej wartości daje sama biblioteka Magenta. Długość każdej podlisty zbioru Y to 1.

Przykład podziału na X oraz Y dla nut liczbowych [145, 7, 131, 31, 342, 1231, 99, 1, 131, 55, 41, 442, 14, 15, 16, 99, 245, 16, 19, 204, 99, 131, 3]:

```
X = [ [145, 7, 131, 31, 342, 1231, 99, 1, 131, 55, 41, 442, 14, 15, 16, 99, 245, 16, 19, 204], [7, 131, 31, 342, 1231, 99, 1, 131, 55, 41, 442, 14, 15, 16, 99, 245, 16, 19, 204, 99], ... ]

Y = [ [99], [131], ... ]
```

Oczywiście postać X i Y jest wcześniej zmapowana do wartości wektorów binarnych.


Dodatkowo ustaliłem tutaj, że dla każdego utworu muzycznego ograniczymy się do wartości bliskiej najmniejszego utworu w datasecie - ustaliłem długość każdego jako 400 nut/akordów.

In [10]:
def generate_dataset(input_array, n_prev = 20):
  temp_x = []
  temp_y = []
  for midi in input_array:
    temp_x.append([midi[i:i+n_prev] for i in range(len(midi) - n_prev) if i+n_prev < 400])
    temp_y.append([midi[i+n_prev] for i in range(len(midi) - n_prev) if i+n_prev < 400])

  temp_x = list(itertools.chain.from_iterable(temp_x))
  temp_y = list(itertools.chain.from_iterable(temp_y))

  return np.array(temp_x), np.array(temp_y)

In [12]:
x, y = generate_dataset(data)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, shuffle=False)

In [ ]:
with open("data/x_train" , "wb") as file:
    pickle.dump(x_train, file)

with open("data/x_test" , "wb") as file:
    pickle.dump(x_test, file)

with open("data/y_train" , "wb") as file:
    pickle.dump(y_train, file)

with open("data/y_test" , "wb") as file:
    pickle.dump(y_test, file)